# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pickle
import re

import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('SELECT * FROM InsertTableName', engine)

In [3]:
# # sampling for debug
# df = df.sample(500)

In [4]:
X = df['message'].values
y = df.iloc[:, 4:].values

### 2. Write a tokenization function to process your text data

In [5]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text, lemmatizer=WordNetLemmatizer()):
    # Detecte URLs
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    
    # Normalize and tokenize
    tokens = nltk.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))
    
    # Remove stopwords
    tokens = [t for t in tokens if t not in stopwords.words('english')]

    # Lemmatize
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,  y)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def classification_scores(y_test, y_pred, labels):
    '''
    return scores of accuracy, f1, precision and recall by DataFrame format.
    '''
    scores = []
    for i, label in enumerate(labels):
        y_t, y_p = y_test[:, i], y_pred[:, i] 
        scores.append([label, 
                       accuracy_score(y_t, y_p), 
                       f1_score(y_t, y_p, average='weighted'),
                       precision_score(y_t, y_p, average='weighted'),
                       recall_score(y_t, y_p, average='weighted')])
    df_scores = pd.DataFrame(scores, columns=['label', 'accuracy', 'f1', 'precision', 'recall'])
    df_scores.set_index('label', inplace=True)
    return df_scores

In [9]:
df_scores = classification_scores(y_test, y_pred, df.columns[4:])
df_scores

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,accuracy,f1,precision,recall
label,,,,
related,0.821941,0.816133,0.813106,0.821941
request,0.879158,0.865350,0.869214,0.879158
offer,0.995117,0.992834,0.990562,0.995117
aid_related,0.745041,0.740874,0.743469,0.745041
medical_help,0.924016,0.898754,0.901711,0.924016
medical_products,0.946750,0.926702,0.931045,0.946750
search_and_rescue,0.974672,0.962615,0.966961,0.974672
security,0.981538,0.972998,0.964605,0.981538
military,0.968874,0.955130,0.956320,0.968874


In [10]:
def print_score(df_scores, score_name):
    '''
    print out socres of min, max and mean.
    '''
    print('[{}]'.format(score_name))
    print('mean : {:.2f}'.format(df_scores[score_name].mean()))
    print('min : {0:.2f} ({1})'.format(df_scores[score_name].min(), df_scores[score_name].idxmin()))
    print('max : {0:.2f} ({1})'.format(df_scores[score_name].max(), df_scores[score_name].idxmax()))
    print('mean : {:.2f}'.format(df_scores[score_name].mean()))
    print('')

In [11]:
print_score(df_scores, 'accuracy')
print_score(df_scores, 'f1')
print_score(df_scores, 'precision')
print_score(df_scores, 'recall')

[accuracy]
mean : 0.94
min : 0.75 (aid_related)
max : 1.00 (child_alone)
mean : 0.94

[f1]
mean : 0.93
min : 0.74 (aid_related)
max : 1.00 (child_alone)
mean : 0.93

[precision]
mean : 0.93
min : 0.74 (aid_related)
max : 1.00 (child_alone)
mean : 0.93

[recall]
mean : 0.94
min : 0.75 (aid_related)
max : 1.00 (child_alone)
mean : 0.94



### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
    'clf__estimator__n_estimators': [10, 100]
}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_weighted', n_jobs=-1)

In [13]:
cv.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_pred = cv.predict(X_test)

In [15]:
df_scores = classification_scores(y_test, y_pred, df.columns[4:])
df_scores

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,accuracy,f1,precision,recall
label,,,,
related,0.834605,0.822950,0.822981,0.834605
request,0.891669,0.880827,0.884809,0.891669
offer,0.995117,0.992834,0.990562,0.995117
aid_related,0.768691,0.767491,0.767326,0.768691
medical_help,0.923863,0.894676,0.902844,0.923863
medical_products,0.948276,0.928352,0.938863,0.948276
search_and_rescue,0.973909,0.961931,0.954061,0.973909
security,0.981843,0.973150,0.964610,0.981843
military,0.968721,0.955046,0.954893,0.968721


In [16]:
print_score(df_scores, 'accuracy')
print_score(df_scores, 'f1')
print_score(df_scores, 'precision')
print_score(df_scores, 'recall')

[accuracy]
mean : 0.95
min : 0.77 (aid_related)
max : 1.00 (child_alone)
mean : 0.95

[f1]
mean : 0.93
min : 0.77 (aid_related)
max : 1.00 (child_alone)
mean : 0.93

[precision]
mean : 0.93
min : 0.77 (aid_related)
max : 1.00 (child_alone)
mean : 0.93

[recall]
mean : 0.95
min : 0.77 (aid_related)
max : 1.00 (child_alone)
mean : 0.95



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
# add features TF=IDF and use algorithm AdaBoost
pipeline_ada = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [18]:
parameters_ada = {
    'clf__estimator__learning_rate': [0.01, 0.1, 0.5],
    'clf__estimator__n_estimators': [10, 100]
}

cv_ada = GridSearchCV(pipeline_ada, param_grid=parameters, scoring='f1_weighted', n_jobs=-1)

In [19]:
cv_ada.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=0)

In [20]:
y_pred = cv.predict(X_test)

In [21]:
df_scores = classification_scores(y_test, y_pred, df.columns[4:])
df_scores

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,accuracy,f1,precision,recall
label,,,,
related,0.834605,0.822950,0.822981,0.834605
request,0.891669,0.880827,0.884809,0.891669
offer,0.995117,0.992834,0.990562,0.995117
aid_related,0.768691,0.767491,0.767326,0.768691
medical_help,0.923863,0.894676,0.902844,0.923863
medical_products,0.948276,0.928352,0.938863,0.948276
search_and_rescue,0.973909,0.961931,0.954061,0.973909
security,0.981843,0.973150,0.964610,0.981843
military,0.968721,0.955046,0.954893,0.968721


In [22]:
print_score(df_scores, 'accuracy')
print_score(df_scores, 'f1')
print_score(df_scores, 'precision')
print_score(df_scores, 'recall')

[accuracy]
mean : 0.95
min : 0.77 (aid_related)
max : 1.00 (child_alone)
mean : 0.95

[f1]
mean : 0.93
min : 0.77 (aid_related)
max : 1.00 (child_alone)
mean : 0.93

[precision]
mean : 0.93
min : 0.77 (aid_related)
max : 1.00 (child_alone)
mean : 0.93

[recall]
mean : 0.95
min : 0.77 (aid_related)
max : 1.00 (child_alone)
mean : 0.95



### 9. Export your model as a pickle file

In [23]:
with open('model.pkl', 'wb') as file:
    pickle.dump(cv_ada, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.